In [35]:
!pip install pandas

astropy 2.0.2 has requirement pytest>=2.8, but you'll have pytest 2.7.1 which is incompatible.
pyasn1-modules 0.2.2 has requirement pyasn1<0.5.0,>=0.4.1, but you'll have pyasn1 0.1.7 which is incompatible.
google-api-core 1.2.1 has requirement requests<3.0.0dev,>=2.18.0, but you'll have requests 2.14.2 which is incompatible.
odo 0.4.2 has requirement toolz>=0.7.3, but you'll have toolz 0.7.2 which is incompatible.


In [34]:
import tensorflow as tf
import numpy
from numpy import genfromtxt
import os
import pandas as pd


ImportError: No module named pandas

In [27]:
os.getcwd()
#os.chdir('./stimuli_params')
os.getcwd()

'/Users/Pam_sf_wang/Documents/perceptual_learning/stimuli/stimuli_params'

In [30]:
df=pd.read_csv('myfile.csv', sep=',',header=None)

.DS_Store
jfj_stimuli_dimension_traintest_070.jpg
jfj_stimuli_dimension_train_020.jpg
jfj_stimuli_dimension_train_030.jpg
jfj_stimuli_dimension_traintest_060.jpg
jfj_stimuli_dimension_traintest_100.jpg
jfj_stimuli_dimension_train_040.jpg
jfj_stimuli_dimension_train_080.jpg
jfj_stimuli_dimension_traintest_010.jpg
jfj_stimuli_dimension_test_100.jpg
jfj_stimuli_dimension_train_090.jpg
jfj_stimuli_param_train.csv
jfj_stimuli_dimension_train_050.jpg
jfj_stimuli_dimension_train_060.jpg
jfj_stimuli_dimension_train_100.jpg
jfj_stimuli_dimension_traintest_030.jpg
jfj_stimuli_dimension_traintest_020.jpg
jfj_stimuli_dimension_train_070.jpg
jfj_stimuli_dimension_traintest_050.jpg
jfj_stimuli_param_test.csv
jfj_stimuli_dimension_traintest_090.jpg
jfj_stimuli_dimension_train_010.jpg
jfj_stimuli_dimension_traintest_080.jpg
jfj_stimuli_dimension_traintest_040.jpg
stimuli_dimension_train_030.jpg
stimuli_dimension_train_020.jpg
stimuli_dimension_train_test_010.jpg
stimuli_param_uniqueF3_test.csv
stimuli

In [14]:
nsamples = 1000

#Data generation, probably a better way to do this
#x_data = numpy.floor(2*numpy.random.random([nsamples,2]))-1
#y_data = numpy.array(map(lambda x: (x[0] or x[1]) and not (x[0] and x[1]),x_data))
#y_data = y_data.reshape([nsamples,1])

test_x_data = numpy.array([[1,1,2.2],[1,1,2.5],[1,1,5.4],[1,3,5.2]])
test_y_data = numpy.array([[1],[1],[0],[0]])

#Just a way of choosing data from this set
data_indices = numpy.random.choice(range(len(test_x_data)),nsamples)
x_data = numpy.array(test_x_data[data_indices]) 
y_data = numpy.array(test_y_data[data_indices])

In [36]:
type(x_data)

numpy.ndarray

In [6]:
#You can think of most the tensorflow variables as actually being functions that we will call when we want to get their value. 
input_ph = tf.placeholder(tf.float32, shape=[3,1]) #This will be the place the input to the network is inserted
target_ph =  tf.placeholder(tf.float32, shape=[1,1]) #This will be the place the target for the network is insertedd
W1 = tf.Variable(tf.random_uniform([10,3],-1,1)) #First layer weights
b1 = tf.Variable(tf.random_uniform([10,1],-1,1)) # " " biases
#[# nodes, # nodes], sampling from a random uniform distribution from -1 to 1 for initial weights
W2 = tf.Variable(tf.random_uniform([1,10],-1,1)) #2nd layer
b2 = tf.Variable(tf.random_uniform([1],-1,1))
h1 = tf.nn.tanh(tf.matmul(W1,input_ph)+b1)
output = tf.nn.tanh(tf.matmul(W2,h1)+b2) #This is the actual construction of the network. When we want to get the output of the network, we will tell tensorflow what to put in the input placeholder, and then we'll run this output function

loss = tf.reduce_sum(tf.square(output - target_ph)) #This is the function we're trying to optimize. The reduce_sum is not really necessary since we only have a single output, just using it to flatten the output.
optimizer = tf.train.AdamOptimizer(0.005) #This is a fancy version of momentum based gradient descent optimization.
train = optimizer.minimize(loss) #This will be how we tell the network to train on an example


In [7]:
# Launch the graph -- tell tensorflow to initialize everything.
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) #first argument to sess.run is function to run, here we're running the initialize function


In [11]:
def test():
    MSE = 0.0
    for i in xrange(len(test_x_data)):
	MSE += sess.run(loss,feed_dict={input_ph: test_x_data[i].reshape([3,1]),target_ph: test_y_data[i].reshape([1,1])}) #test on a test data point. feed_dict is how you pass things in to the placeholders created above
    MSE /= 4
    return MSE

def accuracy():
    ac = []
    for i in xrange(len(test_y_data)):
        ac[i]=sess.run(output,feed_dict={input_ph: test_x_data[i].reshape([3,1])})
    return ac
    
    
def hidden():
    hidden = []
     for i in xrange(len(test_y_data)):
        hidden[i]=sess.run(h1,feed_dict={input_ph: test_x_data[i].reshape([3,1])})
    return hidden
    

# Fit the function
print "Pre training MSE:", test
hi = []
for step in xrange(1000):
    sess.run(train,feed_dict={input_ph: x_data[step % nsamples].reshape([3,1]),target_ph: y_data[step % nsamples].reshape([1,1])}) #Run training on an example
    if step % 100 == 0:
	print "On step %i, test MSE %f" %(step,test())

print "Post training MSE:", test()

print "Final weights:"
print(sess.run(W1),sess.run(b1),sess.run(W2),sess.run(b2))

Pre training MSE: <function test at 0x1173139b0>
On step 0, test MSE 0.000037
On step 100, test MSE 0.000032
On step 200, test MSE 0.000038
On step 300, test MSE 0.000027
On step 400, test MSE 0.000025
On step 500, test MSE 0.000036
On step 600, test MSE 0.001525
On step 700, test MSE 0.000203
On step 800, test MSE 0.000061
On step 900, test MSE 0.000045
Post training MSE: 3.3465088336e-05
Final weights:
(array([[ 1.10700679,  1.36893511,  0.81173515],
       [-0.96622521, -0.44981945,  0.87872887],
       [ 1.08760583,  0.05313268, -0.68299586],
       [-0.64254761,  0.01940801,  0.51178169],
       [-0.56238919,  1.30563736,  0.35752839],
       [-0.28266504, -0.28211924, -0.3925786 ],
       [ 0.91560954, -0.25237444, -0.17506418],
       [ 0.05703747, -0.04092243, -0.32288355],
       [-0.55343038, -1.02382958, -1.17707908],
       [-1.21164322, -1.22286832, -1.26194203]], dtype=float32), array([[ 0.05276117],
       [-0.94872504],
       [ 0.81992096],
       [-0.64839041],
      

In [12]:
hidden

[array([[ 0.99526304]], dtype=float32),
 array([[ 0.99526501]], dtype=float32),
 array([[ 0.99526733]], dtype=float32),
 array([[ 0.99527144]], dtype=float32),
 array([[ 0.99527782]], dtype=float32),
 array([[ 0.99528557]], dtype=float32),
 array([[ 0.99529284]], dtype=float32),
 array([[ 0.99529922]], dtype=float32),
 array([[ 0.99530381]], dtype=float32),
 array([[ 0.99530804]], dtype=float32),
 array([[ 0.99531251]], dtype=float32),
 array([[ 0.99531925]], dtype=float32),
 array([[ 0.99532825]], dtype=float32),
 array([[ 0.99533874]], dtype=float32),
 array([[ 0.99534899]], dtype=float32),
 array([[ 0.9953528]], dtype=float32),
 array([[ 0.99535239]], dtype=float32),
 array([[ 0.9953503]], dtype=float32),
 array([[ 0.99535155]], dtype=float32),
 array([[ 0.99535501]], dtype=float32),
 array([[ 0.99535882]], dtype=float32),
 array([[ 0.99536484]], dtype=float32),
 array([[ 0.99537081]], dtype=float32),
 array([[ 0.99537879]], dtype=float32),
 array([[ 0.9953869]], dtype=float32),
 ar